Importação das Bibliotecas

In [10]:
import os
import zipfile
import shutil
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import concurrent.futures

Diretório das Imagens e Saídas de .CSV e DataFrame

In [19]:
# Treino TrabalhoSentinel-2
# origem = "/home/emanoelcepr/projetos/u-net/TESTE/"
# diretorio_imagens = "/home/emanoelcepr/projetos/u-net/TESTE_DESCOMPACTADO/"
# banda11 = "/home/emanoelcepr/projetos/u-net/teste_banda11"
# banda12 = "/home/emanoelcepr/projetos/u-net/teste_banda12"
# mask = "/home/emanoelcepr/projetos/u-net/teste_mask"

# Treino Sentinel-2
origem = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/treino/"
diretorio_imagens = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/treino_processados/"
banda11 = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/banda11"
banda12 = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/banda12"
mask = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/mask"
wildfires = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/wildfires"

# Teste Sentinel-2
# origem = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/teste/"
# diretorio_imagens = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/teste_processado/"
# banda11 = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/teste_banda11"
# banda12 = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/teste_banda12"
# mask = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/teste_mask"
# Teste Serra do Cipó Sentinel-2
# origem = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/cipo/"
# diretorio_imagens = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/cipo_processado/"
# banda11 = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/cipo_banda11"
# banda12 = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/cipo_banda12"
# mask = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/cipo_mask"
# Teste Landsat-8
#origem = "/home/cristiano/Imagens/imagens_originais/Landsat-8/teste/"
#diretorio_imagens = "/home/cristiano/Imagens/imagens_originais/Landsat-8/teste_processado/"

Extrair as imagens para o diretório "treino_processado" e adicionando um número para diferenciar as imagens de mesmo nome.

In [14]:
numero_imagem = 1

# Verifica se o diretório de destino existe, caso contrário, cria-o
if not os.path.exists(diretorio_imagens):
    os.makedirs(diretorio_imagens)

# Percorre todas as pastas compactadas no diretório de origem
for pasta_compactada in os.listdir(origem):
    caminho_pasta_compactada = os.path.join(origem, pasta_compactada)

    # Verifica se o item é uma pasta compactada
    if zipfile.is_zipfile(caminho_pasta_compactada):
        # Extrai o conteúdo do arquivo compactado para uma pasta temporária
        pasta_temporaria = os.path.join(diretorio_imagens, "temp")
        with zipfile.ZipFile(caminho_pasta_compactada, 'r') as zip_ref:
            zip_ref.extractall(pasta_temporaria)

        # Percorre todas as imagens na pasta temporária
        for nome_imagem in os.listdir(pasta_temporaria):
            caminho_imagem = os.path.join(pasta_temporaria, nome_imagem)

            # Verifica se o item é um arquivo de imagem
            if os.path.isfile(caminho_imagem):
                # Define o novo nome da imagem com o número no início
                novo_nome_imagem = f"{numero_imagem}_{nome_imagem}"

                # Move a imagem para o diretório de destino com o novo nome
                caminho_destino_imagem = os.path.join(diretorio_imagens, novo_nome_imagem)
                shutil.move(caminho_imagem, caminho_destino_imagem)

                # Incrementa o número para a próxima imagem
                numero_imagem += 1

        # Remove a pasta temporária após processar todas as imagens
        shutil.rmtree(pasta_temporaria)

print("Descompactação concluída.")

Descompactação concluída.


Cria as Máscaras

In [15]:
def modificar_nome_arquivo(nome_arquivo):
    # Remover a sequência "_(Raw)"
    nome_arquivo_modificado = nome_arquivo.replace("(Raw)", "")
    return nome_arquivo_modificado

def limiar(diretorio):
    # Percorre todas as imagens no diretório
    for nome_arquivo in os.listdir(diretorio):
        if "B11_(Raw)" in nome_arquivo:
            # Limpar o nome do arquivo
            nome_arquivo_limpo = modificar_nome_arquivo(nome_arquivo)

            # Carrega a imagem
            caminho_arquivo = os.path.join(diretorio, nome_arquivo)
            imagem = cv2.imread(caminho_arquivo, cv2.IMREAD_GRAYSCALE)

            # Aplicar o limiar de 140
            limiar = 140
            _, thresh = cv2.threshold(imagem, limiar, 1, cv2.THRESH_BINARY)

            # Obter o nome do arquivo sem a extensão
            nome_sem_extensao = os.path.splitext(nome_arquivo_limpo)[0]

            # Salvar a imagem limiarizada com "_Mask" antes da extensão no mesmo diretório
            nome_mask = nome_sem_extensao + "Mask.tiff"
            caminho_mask = os.path.join(diretorio, nome_mask)

            cv2.imwrite(caminho_mask, thresh * 255)

        elif "B12_(Raw)" in nome_arquivo:
            # Limpar o nome do arquivo
            nome_arquivo_limpo = modificar_nome_arquivo(nome_arquivo)

            # Carrega a imagem
            caminho_arquivo = os.path.join(diretorio, nome_arquivo)
            imagem = cv2.imread(caminho_arquivo, cv2.IMREAD_GRAYSCALE)

            # Aplicar o limiar de 180
            limiar = 180
            _, thresh = cv2.threshold(imagem, limiar, 1, cv2.THRESH_BINARY)

            # Obter o nome do arquivo sem a extensão
            nome_sem_extensao = os.path.splitext(nome_arquivo_limpo)[0]

            # Salvar a imagem limiarizada com "_Mask" antes da extensão no mesmo diretório
            nome_mask = nome_sem_extensao + "Mask.tiff"
            caminho_mask = os.path.join(diretorio, nome_mask)

            cv2.imwrite(caminho_mask, thresh * 255)

    # Libera a memória das imagens
    cv2.destroyAllWindows()

limiar(diretorio_imagens)

Recorte das imagens em 64x64

In [16]:
tamanho_recorte = 64
numero_imagem = 1

def recortar_imagens(diretorio, tamanho_recorte):
    global numero_imagem
    
    # Percorre todas as imagens no diretório
    for nome_arquivo in os.listdir(diretorio):
        if nome_arquivo.endswith(".tiff"):
            # Carrega a imagem
            caminho_arquivo = os.path.join(diretorio, nome_arquivo)
            imagem = cv2.imread(caminho_arquivo)
            
            # Obtém as dimensões da imagem
            altura, largura, _ = imagem.shape
            
            # Calcula o número de recortes horizontais e verticais
            num_recortes_horizontais = largura // tamanho_recorte
            num_recortes_verticais = altura // tamanho_recorte
            
            # Realiza o recorte e salva as imagens resultantes
            for i in range(num_recortes_horizontais):
                for j in range(num_recortes_verticais):
                    # Define as coordenadas do retângulo de recorte
                    esquerda = i * tamanho_recorte
                    superior = j * tamanho_recorte
                    direita = (i + 1) * tamanho_recorte
                    inferior = (j + 1) * tamanho_recorte
                    
                    # Realiza o recorte
                    recorte = imagem[superior:inferior, esquerda:direita]
                    
                    # Separa o nome do arquivo e a extensão
                    nome, extensao = os.path.splitext(nome_arquivo)
                    
                    numero = nome_arquivo.split("_")[0]
                   
                    nome = nome[nome.index('_'): len(nome)]

                    # Constrói o novo nome de arquivo com o prefixo desejado
                   
                    novo_nome_arquivo = f"{str(numero).zfill(6)}{nome}_recorte_{i}_{j}{extensao}"
                    
                    # Salva o recorte em um novo arquivo
                    caminho_recorte = os.path.join(diretorio, novo_nome_arquivo)
                    cv2.imwrite(caminho_recorte, recorte)
                    
                    # Incrementa o número para a próxima imagem
                    numero_imagem += 1
                    
            # Remove a imagem original
            os.remove(caminho_arquivo)
            
    # Libera a memória das imagens
    cv2.destroyAllWindows()

recortar_imagens(diretorio_imagens, tamanho_recorte)


Normalizar imagens

In [17]:
def normalize_images(input_dir):
    for filename in os.listdir(input_dir):
        if 'Mask' in filename:
            input_path = os.path.join(input_dir, filename)
            img = Image.open(input_path)

            # Converte a imagem para modo 'L', que é escala de cinza
            img = img.convert('L')

            # Normaliza os valores dos pixels dividindo por 255 e convertendo para inteiro
            img = img.point(lambda x: int(x / 255))

            # Salva a imagem normalizada substituindo a original
            img.save(input_path)

normalize_images(diretorio_imagens)


In [18]:
# Verificar se os diretórios de destino existem e criar se não existirem
for diretorio in [banda11, banda12, mask]:
    if not os.path.exists(diretorio):
        os.makedirs(diretorio)

# Listar os arquivos no diretório de origem
arquivos = os.listdir(diretorio_imagens)

for arquivo in arquivos:
    caminho_arquivo = os.path.join(diretorio_imagens, arquivo)

    if 'B11_(Raw)' in arquivo:
        shutil.move(caminho_arquivo, os.path.join(banda11, arquivo))
    elif 'B12_(Raw)' in arquivo:
        shutil.move(caminho_arquivo, os.path.join(banda12, arquivo))
    elif 'B12_Mask' in arquivo:
        shutil.move(caminho_arquivo, os.path.join(mask, arquivo))
    elif 'Wildfires' in arquivo:
        shutil.move(caminho_arquivo, os.path.join(wildfires, arquivo))    

